In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, BasicAer, transpile
from math import pi, sqrt

In [2]:
def inv_qft_gate(nbdigits):
    qr = QuantumRegister(nbdigits, 'qr')
    circuit = QuantumCircuit(qr)
    for i in range(nbdigits // 2):
        circuit.swap(i, nbdigits-i-1)
    for i in range(nbdigits):
        circuit.h(qr[i])
        for j in range(i+1, nbdigits):
            rot_phase = - 2*pi/2**(j-i+1)
            circuit.cp(rot_phase, j, i)
    return circuit.to_gate()

In [3]:
nbdigits = 5
theta = 0.825

In [4]:
initial_state = np.zeros(2**(nbdigits+1), dtype=np.complex_)
initial_state[0] = 1. / sqrt(2)
initial_state[2**nbdigits] = np.exp(2*pi*theta*1.j) / sqrt(2)

In [5]:
circuit = QuantumCircuit(nbdigits+1, nbdigits)
circuit.initialize(initial_state)
circuit.h(range(nbdigits))
for i in range(nbdigits):
    circuit.cp(2*pi*2**(nbdigits-i-1), i, nbdigits)
circuit.append(inv_qft_gate(nbdigits), range(nbdigits))
circuit.measure(range(nbdigits), range(nbdigits))

In [6]:
# circuit.draw()

In [7]:
backend = Aer.get_backend('aer_simulator')
tqc = transpile(circuit, backend)
job = backend.run(tqc, shots=1000)
result = job.result()
counts = result.get_counts()
print(counts)

{'00000': 1000}


In [8]:
circuit1 = QuantumCircuit(nbdigits+1, nbdigits)
circuit1.initialize(initial_state)
circuit1.h(range(nbdigits))
for i in range(nbdigits):
    circuit1.cp(2*pi*2**(nbdigits-i-1), i, nbdigits)
circuit1.append(inv_qft_gate(nbdigits), range(nbdigits))

In [9]:
backend = BasicAer.get_backend('statevector_simulator')
tqc1 = transpile(circuit1, backend)
job = backend.run(tqc1)
result = job.result()
statevector = result.get_statevector(tqc1)

In [11]:
np.where(statevector!=0)

(array([ 0, 32]),)

In [12]:
statevector[0]

(0.707106781186546-1.5264700697611401e-15j)

In [13]:
statevector[32]

(0.32101976096010093-0.6300367553350494j)